<a href="https://colab.research.google.com/github/bhussn/SecSplitLLM/blob/main/SecSplitLLM/notebooks/bert/Benchmark_Bert_SST_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install -q transformers datasets evaluate wandb
!pip install -U datasets

In [ ]:
# Imports and W&B login
import os
import time
import wandb
import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments,
    TrainerCallback
)
import evaluate

# Login to W&B
wandb.login()

In [ ]:
#  Load SST-2 Dataset
dataset = load_dataset("glue", "sst2")

# Use a subset of the dataset (did 10%)
dataset["train"] = dataset["train"].shuffle(seed=42).select(range(int(0.1 * len(dataset["train"]))))
dataset["validation"] = dataset["validation"].shuffle(seed=42).select(range(int(0.1 * len(dataset["validation"]))))

# Tokenizer and Preprocessing
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Load Pretrained BERT for Sequence Classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

In [ ]:
# Define Accuracy Metric
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=preds, references=labels)

# TrainingArguments with W&B integration
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    warmup_steps=100,
    weight_decay=0.01,
    save_strategy="epoch",
    eval_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    report_to="wandb",
    run_name="bert-sst2-finetune"
)

In [ ]:
# Track Timer per epoch
class SimpleTimerCallback(TrainerCallback):
    def __init__(self):
        self.epoch_start_time = None

    def on_epoch_begin(self, args, state, control, **kwargs):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, args, state, control, **kwargs):
        epoch_duration = time.time() - self.epoch_start_time
        print(f"Epoch {int(state.epoch)} duration: {epoch_duration:.2f} seconds")
        wandb.log({f"epoch_{int(state.epoch)}_duration_sec": epoch_duration})

In [ ]:
# Track Training Time
start_time = time.time()

# Trainer Setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics,
    callbacks=[SimpleTimerCallback()]
)

# Start Training
trainer.train()

# Evaluate (seperate)
results = trainer.evaluate()
print(f"Validation Results: {results}")

# Log training time and GPU usage
end_time = time.time()
train_time = end_time - start_time
print(f"\n Training Time (seconds): {train_time}")

# Show GPU memory usage
print("\n GPU Memory Usage:")
!nvidia-smi

# Log to W&B
wandb.log({"train_time_seconds": train_time})
wandb.log(results)

# Save model
model.save_pretrained("bert-finetuned-sst2")
tokenizer.save_pretrained("bert-finetuned-sst2")
